In [3]:
from pytket import Circuit

circ = Circuit(2,2) # define a circuit with 2 qubits and 2 bits
circ.H(0)           # add a Hadamard gate to qubit 0
circ.Rz(0.25, 0)    # add an Rz gate of angle 0.25*pi to qubit 0
circ.CX(1,0)        # add a CX gate with control qubit 1 and target qubit 0
circ.measure_all()  # measure qubits 0 and 1, recording the results in bits 0 and 1

[H q[0]; Rz(0.25) q[0]; CX q[1], q[0]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; ]